In [1]:
import os, sys
import numpy as np
# path to access c++ files
installation_path = os.getenv("INSTALL_PATH")
sys.path.append(installation_path)

In [2]:
from cunqa import getQPUs

qpus  = getQPUs()

for q in qpus:
    print(f"QPU {q.id}, backend: {q.backend.name}, simulator: {q.backend.simulator}, version: {q.backend.version}.")


QPU 0, backend: BasicMunich, simulator: MunichSimulator, version: 0.0.1.
QPU 1, backend: BasicMunich, simulator: MunichSimulator, version: 0.0.1.
QPU 2, backend: BasicMunich, simulator: MunichSimulator, version: 0.0.1.
QPU 3, backend: BasicMunich, simulator: MunichSimulator, version: 0.0.1.
QPU 4, backend: BasicMunich, simulator: MunichSimulator, version: 0.0.1.
QPU 5, backend: BasicMunich, simulator: MunichSimulator, version: 0.0.1.
QPU 6, backend: BasicMunich, simulator: MunichSimulator, version: 0.0.1.
QPU 7, backend: BasicMunich, simulator: MunichSimulator, version: 0.0.1.
QPU 8, backend: BasicMunich, simulator: MunichSimulator, version: 0.0.1.
QPU 9, backend: BasicMunich, simulator: MunichSimulator, version: 0.0.1.
QPU 10, backend: BasicMunich, simulator: MunichSimulator, version: 0.0.1.
QPU 11, backend: BasicMunich, simulator: MunichSimulator, version: 0.0.1.
QPU 12, backend: BasicMunich, simulator: MunichSimulator, version: 0.0.1.
QPU 13, backend: BasicMunich, simulator: MunichS

# Paralelization for gradient-free optimizers: Differential Evolution

_Introduction and explanation_

We recover the variational circuit used before:

In [3]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter

def hardware_efficient_ansatz(num_qubits, num_layers):
    qc = QuantumCircuit(num_qubits)
    param_idx = 0
    for _ in range(num_layers):
        for qubit in range(num_qubits):
            phi = Parameter(f'phi_{param_idx}_{qubit}')
            lam = Parameter(f'lam_{param_idx}_{qubit}')
            qc.ry(phi, qubit)
            qc.rz(lam, qubit)
        param_idx += 1
        for qubit in range(num_qubits - 1):
            qc.cx(qubit, qubit + 1)
    qc.measure_all()
    return qc

In [4]:
def target_distribution(num_qubits):
    # Define a normal distribution over the states
    num_states = 2 ** num_qubits
    states = np.arange(num_states)
    mean = num_states / 2
    std_dev = num_states / 4
    target_probs = norm.pdf(states, mean, std_dev)
    target_probs /= target_probs.sum()  # Normalize to make it a valid probability distribution
    target_dist = {format(i, f'0{num_qubits}b'): target_probs[i] for i in range(num_states)}
    return target_dist

import pandas as pd
from scipy.stats import entropy, norm

def KL_divergence(counts, n_shots, target_dist):
    # Convert counts to probabilities
    pdf = pd.DataFrame.from_dict(counts, orient="index").reset_index()
    pdf.rename(columns={"index": "state", 0: "counts"}, inplace=True)
    pdf["probability"] = pdf["counts"] / n_shots
    
    # Create a dictionary for the obtained distribution
    obtained_dist = pdf.set_index("state")["probability"].to_dict()
    
    # Ensure all states are present in the obtained distribution
    for state in target_dist:
        if state not in obtained_dist:
            obtained_dist[state] = 0.0
    
    # Convert distributions to lists for KL divergence calculation
    target_probs = [target_dist[state] for state in sorted(target_dist)]
    obtained_probs = [obtained_dist[state] for state in sorted(obtained_dist)]
    
    # Calculate KL divergence
    kl_divergence = entropy(obtained_probs, target_probs)
    
    return kl_divergence
    

In [5]:
num_qubits = 6

num_layers = 3

n_shots = 100000

target_dist = target_distribution(num_qubits)

In [6]:
def cost_function(result):
    
    global target_dist
    
    counts = result.get_counts()
    
    return KL_divergence(counts, n_shots, target_dist)

In [7]:
ansatz = hardware_efficient_ansatz(num_qubits, num_layers)

num_parameters = ansatz.num_parameters

initial_parameters = np.zeros(num_parameters)

In [8]:
init_qjobs = []
init_params = np.zeros(num_parameters)
for q in qpus:
    init_qjobs.append(q.run(ansatz.assign_parameters(init_params), transpile=False, shots=n_shots))

from cunqa import QJobMapper
mapper = QJobMapper(init_qjobs)

In [9]:
pop=[]
total_pop=1*num_parameters
for j in range(total_pop):
    initial_point=np.random.uniform(-np.pi, np.pi, num_parameters)
    pop.append(initial_point)

bounds=[]
for i in range(0,num_parameters):
    bounds.append((-np.pi,np.pi))

print("Bounds:", len(bounds))
print("Initial population:", len(pop))

best_individual = []

def cb(xk,convergence=1e-8):
     best_individual.append(xk)

from scipy.optimize import differential_evolution
import time

tick = time.time()
result = differential_evolution(cost_function, bounds, maxiter=500, disp=True, workers=mapper, strategy='best1bin', init=pop, polish = False, callback=cb)
tack = time.time()
print(result)

energies = mapper(cost_function, best_individual)



print("Time:", tack-tick)

Bounds: 36
Initial population: 36
/mnt/netapp1/Store_CESGA/home/cesga/mlosada/api/api-simulator/installation/cunqa/qjob.py:81
	error: Error during simulation, please check availability of QPUs, run arguments sintax and circuit sintax: Error updating qasm parameters.
/mnt/netapp1/Store_CESGA/home/cesga/mlosada/api/api-simulator/installation/cunqa/qjob.py:381
	error: Error while creating Results object [QJobError]


/opt/cesga/qmio/hpc/software/Compiler/gcc/12.3.0/scipy/1.13.0-python-3.9.9/lib/python3.9/site-packages/scipy/optimize/_differentialevolution.py:487: UserWarning: differential_evolution: the 'workers' keyword has overridden updating='immediate' to updating='deferred'
  with DifferentialEvolutionSolver(func, bounds, args=args,


QJobError: 

In [10]:
result.x

array([-0.73469769,  1.17418177, -1.45503937, -2.20013398, -0.47013537,
       -2.22694554, -2.12828819, -1.42853008, -0.08819974, -1.36653995,
       -1.32538102, -0.87326058, -1.69217034, -2.09175855,  0.33660399,
        3.11309142, -0.0447022 ,  0.12230347,  0.08543365, -0.45315688,
        1.95021447,  1.35692285,  0.59755266,  0.49048743,  1.92368871,
       -0.59651135,  1.41554646,  0.46922337,  1.54441668, -2.60716695,
       -0.73254089, -0.86024541, -1.45114305, -2.49992884, -1.43344119,
       -0.43577026])

In [11]:
result.fun

0.1867568794416486

In [12]:
result.population_energies

array([0.18675688, 0.30652199, 0.33058087, 0.30690289, 0.3275122 ,
       0.30742431, 0.22954697, 0.30879715, 0.28473121, 0.27076335,
       0.3048595 , 0.29648775, 0.27187809, 0.29838232, 0.25908089,
       0.3163563 , 0.29306996, 0.22312884, 0.25812829, 0.22867409,
       0.26686759, 0.32642521, 0.26276147, 0.25293143, 0.27340118,
       0.28361295, 0.28241051, 0.2929356 , 0.23705092, 0.27105013,
       0.27116754, 0.30652199, 0.33058087, 0.30690289, 0.3275122 ,
       0.30742431])

In [13]:
circ = ansatz.assign_parameters(result.x)

In [18]:
for i in range(10):
    print(qpus[0].backend.name)
    counts = qpus[0].run(circ, shots = n_shots, seed = 8).result()
    print(counts)

BasicAer
BasicAer
BasicAer
BasicAer
BasicAer
BasicAer
BasicAer
BasicAer
BasicAer
BasicAer


In [22]:
from qiskit import QuantumCircuit
qc = QuantumCircuit(2)
qc.rx(1.77, 0)
qc.measure_all()

for i in range(10):
    print(qpus[0].backend.name)
    counts = qpus[0].run(qc, shots = n_shots, seed = 8).result().get_counts()
    print(counts)

BasicAer
{'00': 99, '01': 1034, '10': 1039, '11': 826}
BasicAer
{'00': 73, '01': 996, '10': 1024, '11': 821}
BasicAer
{'00': 104, '01': 1001, '10': 958, '11': 868}
BasicAer
{'00': 82, '01': 1056, '10': 970, '11': 742}
BasicAer
{'00': 79, '01': 1066, '10': 1001, '11': 796}
BasicAer
{'00': 93, '01': 1026, '10': 977, '11': 768}
BasicAer
{'00': 97, '01': 989, '10': 985, '11': 893}
BasicAer
{'00': 83, '01': 1004, '10': 973, '11': 801}
BasicAer
{'00': 87, '01': 975, '10': 930, '11': 848}
BasicAer
{'00': 79, '01': 988, '10': 1028, '11': 847}


In [14]:
import matplotlib.pyplot as plt
plt.clf()
plt.plot(np.linspace(0, result.nit, result.nit), energies, label="Optimization path (run())")
upper_bound = result.nit
plt.plot(np.linspace(0, upper_bound, upper_bound), np.zeros(upper_bound), "--", label="Target cost")
plt.xlabel("Step"); plt.ylabel("Cost"); plt.legend(loc="upper right"); plt.title(f"n = {num_qubits}, l = {num_layers}, # params = {num_parameters}")
plt.grid(True)
plt.show()
plt.savefig(f"optimization_de_n_{num_qubits}_p_{num_parameters}.png", dpi=200)

# Paralelization of expectation value terms

In [25]:
# TODO

# Paralelization for gradient optimizers

In [26]:
# TODO